# 最简 vertext index

## 创建索引

In [1]:
%%time

# embedding
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings

ollama_embedding = OllamaEmbedding(
    # model_name="dztech/bge-large-zh:v1.5",
    model_name="chatfire/bge-m3:q8_0",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 3.32 s, sys: 467 ms, total: 3.79 s
Wall time: 3.62 s


In [2]:
%%time

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["../content.txt"]
).load_data()
index = VectorStoreIndex.from_documents(documents)

CPU times: user 300 ms, sys: 24.9 ms, total: 324 ms
Wall time: 1.82 s


In [3]:
index

## 查询

In [4]:
%%time

from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(model="qwen2-7b-6k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1,
                 request_timeout=60.0
                )

Settings.llm =llm

CPU times: user 9.47 ms, sys: 0 ns, total: 9.47 ms
Wall time: 8.97 ms


In [5]:
%%time

query_engine = index.as_query_engine(streaming=True,)
response = query_engine.query("比较亚洲和欧洲有哪些区别？")
response.print_response_stream()

亚洲与欧洲相比，在人口规模上具有显著优势，并且在活力和发展方面展现出强劲势头。亚洲不仅拥有世界上最多的人口，而且其经济、文化以及社会活动都显示出极高的动态性和创新性。

历史背景也显示了两地区之间的差异。亚洲经历了不同的发展路径和挑战，例如冷战时期与欧洲的对比。在和平建设方面，亚洲领导人被期望能够以更加务实、坚持不懈和负责任的方式共同努力，这可能比他们的欧洲同行做得更好。此外，在处理国家主权问题上，亚洲城市如哈尔滨展示了对历史遗产的重视和保护，这是对欧洲殖民主义影响的一种反思。

经济层面，亚洲有机会在经济发展领域为欧洲提供经验教训。随着全球化和技术进步的发展，亚洲地区在经济增长、市场整合以及与邻国的合作方面展现出潜力。

总体而言，亚洲与欧洲之间的区别体现在人口规模、活力、历史背景、和平建设的期望以及经济发展的机会上。CPU times: user 345 ms, sys: 37.9 ms, total: 383 ms
Wall time: 5.16 s


## 存储索引 - qdrant

In [6]:
%%time

import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client.models import Distance, VectorParams
from llama_index.core import StorageContext

CPU times: user 762 ms, sys: 19.8 ms, total: 782 ms
Wall time: 814 ms


In [8]:
%%time

client = qdrant_client.QdrantClient(
    # location=":memory:",
    host="ape", port=6333,
    # vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

CPU times: user 48.1 ms, sys: 0 ns, total: 48.1 ms
Wall time: 47.2 ms


In [9]:
%%time
vector_store = QdrantVectorStore(client=client, collection_name="simple")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

CPU times: user 116 ms, sys: 428 µs, total: 116 ms
Wall time: 3.73 s


## 使用存储索引查询 - qdrant

In [10]:
%%time

query_engine = index.as_query_engine(streaming=True,)
response = query_engine.query("比较亚洲和欧洲有哪些区别？")
response.print_response_stream()

亚洲与欧洲相比，在人口规模上具有显著优势，并且展现出极高的活力和发展潜力。亚洲不仅拥有庞大的人口基数，而且在经济发展、和平建设方面也具备给欧洲提供经验教训的可能性。

历史背景是两者之间的一个关键区别。欧洲经历了冷战的直接冲突和结束阶段，而亚洲地区则没有经历类似的大规模军事对抗。这使得亚洲有机会在和平与稳定的基础上发展经济和社会结构。

此外，亚洲国家之间的合作框架，如上海合作组织（SCO）、金砖国家机制（BRICS）以及亚太经合组织（APEC），为区域内的安全和合作提供了多边平台。这些机构虽然不是解决所有安全挑战的唯一途径，但它们的存在有助于避免陷入欧洲冷战时期的困境。

在经济领域，亚洲在过去几十年中实现了快速增长，并且在国际贸易、制造业和服务业方面取得了显著成就。相比之下，欧洲在这一时期经历了不同的发展轨迹，包括一体化进程和经济结构转型。

最后，亚洲领导人强调通过一系列精心安排的渐进步骤来实现共同目标，如构建“人类命运共同体”，这与欧洲历史上的决策方式形成对比。这种务实、坚持不懈和负责任的合作态度被视为亚洲的一个优势。

总之，亚洲在人口规模、经济发展速度、合作框架以及和平建设方面与欧洲存在显著区别，并且这些差异为亚洲提供了独特的机会来引领全球发展。CPU times: user 388 ms, sys: 30.7 ms, total: 419 ms
Wall time: 9 s


## 加载存储索引 - qdrant

In [11]:
%%time

vector_store = QdrantVectorStore(client=client, collection_name="simple")

storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

CPU times: user 0 ns, sys: 2.96 ms, total: 2.96 ms
Wall time: 4.27 ms


In [12]:
%%time

query_engine = index.as_query_engine(streaming=True,)
response = query_engine.query("比较亚洲和欧洲有哪些区别？")
response.print_response_stream()

亚洲与欧洲之间存在多方面的差异：

1. **人口规模**：亚洲不仅是世界上人口最多的地区，其多样性体现在文化、语言、宗教以及经济发展水平上。相比之下，欧洲在地理范围和人口数量上都较小。

2. **历史背景**：亚洲的历史中包含了丰富的多元文明和帝国体系，如中国、印度、伊斯兰世界等，而欧洲则经历了从古罗马帝国到中世纪、文艺复兴直至现代国家的转变。

3. **经济发展**：文中提到，在经济发展方面，亚洲有机会给欧洲提供经验教训。这可能指的是亚洲在经济增长速度、新兴市场发展以及全球化参与度上的表现。

4. **政治结构**：亚洲地区内存在多种多边合作框架，如上海合作组织（SCO）、金砖国家机制（BRICS）和亚太经合组织（APEC），这些机构反映了区域内的多样化需求和合作模式。欧洲则通过欧盟等一体化组织实现了更深层次的政治经济整合。

5. **冷战后时代**：文中提到亚洲有机会在和平建设方面给欧洲上一课，这可能指的是亚洲国家在处理冲突、促进地区稳定以及构建多边关系方面的经验。

6. **历史记忆与遗产保护**：以哈尔滨为例，亚洲城市对历史遗产的保护和再利用显示出对文化遗产的重视。这与欧洲一些城市的历史保护工作相呼应，但亚洲在这方面可能更加注重恢复和展示其独特的文化身份。

这些比较强调了亚洲和欧洲在地理、人口、历史、经济发展以及政治结构等方面的显著差异，并突出了亚洲在某些领域可能提供给欧洲的经验和教训。CPU times: user 436 ms, sys: 53.9 ms, total: 490 ms
Wall time: 7.92 s


## 存储引擎追加文档 - qdrant

In [13]:
%%time

documents = SimpleDirectoryReader(
    input_files=["../content-short.txt"]
).load_data()

CPU times: user 1.1 ms, sys: 0 ns, total: 1.1 ms
Wall time: 1 ms


In [14]:
%%time

from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=1024, chunk_overlap=10)
new_nodes = parser.get_nodes_from_documents(documents)
index.insert_nodes(new_nodes)

CPU times: user 79.2 ms, sys: 1.27 ms, total: 80.5 ms
Wall time: 907 ms


In [15]:
%%time

query_engine = index.as_query_engine(streaming=True,)
response = query_engine.query("巴基斯坦有哪三大优势？")
response.print_response_stream()

根据提供的上下文信息，文中并未明确提到巴基斯坦具有哪三大优势。因此，无法直接从给定的信息中提取出关于巴基斯坦的三大优势的答案。需要更多具体或相关的上下文来回答这个问题。CPU times: user 82 ms, sys: 3.28 ms, total: 85.3 ms
Wall time: 1.98 s


In [22]:
%%time

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(streaming=True,)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.1)],
)

response = query_engine.query("巴基斯坦有哪三大优势？")
response.print_response_stream()

在当前亚洲安全环境背景下，分析巴基斯坦的优势时，我们可以从以下几个方面进行探讨：

1. **区域合作与影响力**：作为连接南亚、中亚和中东地区的桥梁，巴基斯坦在区域合作中扮演着重要角色。其与邻国如阿富汗、印度以及与中国等大国的紧密关系，为解决地区安全问题提供了多边合作的可能性。

2. **经济潜力与投资机会**：随着“一带一路”倡议的推进，巴基斯坦获得了大量来自中国的投资和项目支持，尤其是通过中巴经济走廊（CPEC）这一旗舰项目。这不仅促进了基础设施建设、工业发展和能源供应，也为吸引国际投资提供了新的机遇。

3. **战略位置与地缘政治**：作为南亚地区的关键国家，巴基斯坦的战略地理位置使其在维护区域稳定方面具有独特优势。其在阿富汗问题上的角色，以及与美国等域外大国的关系，为解决地区冲突和安全挑战提供了可能的途径。

4. **文化与教育交流**：通过加强与亚洲其他国家的文化、教育和人文交流，巴基斯坦能够促进软实力的发展，增强国际形象，并为经济和社会发展创造有利条件。这有助于吸引外国游客、学生和其他投资者，进一步推动国家的多元化发展。

5. **多边合作框架**：参与诸如上海合作组织（SCO）等区域组织，巴基斯坦可以与俄罗斯和中国等成员国加强合作，在安全、贸易、能源等领域实现共同利益最大化。通过这些平台，巴基斯坦能够更好地应对地区内的挑战，并在国际舞台上增强自身的影响力。

综上所述，巴基斯坦的优势在于其独特的地理位置、经济潜力、多边合作关系以及文化软实力的提升，这些都是推动国家发展和解决亚洲地区安全问题的关键因素。CPU times: user 1.3 s, sys: 37.6 ms, total: 1.34 s
Wall time: 42.3 s


In [32]:
%%time

query_engine = index.as_query_engine(streaming=True,verbose=True,)
response = query_engine.query("比较亚洲和欧洲有哪些区别？")
response.print_response_stream()

亚洲与欧洲在多方面存在显著的区别：

1. **人口规模**：亚洲是世界上人口最多的地区，拥有庞大的人口基数。

2. **活力与多样性**：亚洲被描述为最有活力、最有朝气的大陆之一，这反映了其文化、经济和政治的多样性和动态性。

3. **经济发展**：亚洲在经济发展方面展现出巨大潜力，并且在某些领域（如科技、制造业）取得了显著成就。相比之下，欧洲在过去可能在一些传统工业领域领先，但近年来也面临着转型与创新的压力。

4. **历史背景**：亚洲经历了不同的历史阶段和挑战，包括冷战时期的影响较小，而欧洲则经历了冷战的直接冲突。亚洲领导人被期望能够以更务实、坚持不懈和负责任的方式共同合作，以在和平建设方面超越其欧洲同行。

5. **地缘政治关系**：亚洲国家之间的多边框架（如上海合作组织、金砖国家机制、亚太经合组织）强调安全合作，这与欧洲的多边主义有所不同。亚洲领导人被鼓励通过一系列精心安排的渐进步骤，共同迈向“人类命运共同体”。

6. **地区冲突和稳定**：亚洲在处理地区冲突时面临挑战，如印度与周边国家的关系紧张（包括领土争端、民族主义情绪等），而阿富汗局势的稳定对中巴关系至关重要。相比之下，欧洲在过去经历了直接的军事冲突。

7. **恐怖主义威胁**：亚洲和欧洲都受到恐怖主义的威胁，但采取了不同的策略来应对这一挑战。中国在铲除恐怖主义土壤方面取得了成功，并实施了严格的枪支管制政策以减少恐怖分子的立足空间。而美国等国家则面临不同形式的人权与持枪自由之间的平衡。

这些区别反映了亚洲和欧洲在全球政治、经济和社会发展方面的独特路径和发展重点。CPU times: user 466 ms, sys: 61.1 ms, total: 527 ms
Wall time: 13 s


In [30]:
collections = client.get_collections()
collections.collections

[CollectionDescription(name='test'), CollectionDescription(name='simple')]

In [29]:
collections.collections

[CollectionDescription(name='test'), CollectionDescription(name='simple')]

In [31]:
collection_name = "news_collection"

# 检查集合是否存在
if any(collection.name == collection_name for collection in collections.collections):
    print(f"Collection '{collection_name}' exists.")
else:
    print(f"Collection '{collection_name}' does not exist.")

Collection 'news_collection' does not exist.
